# Consumer Loan Pricing

In [1]:
import loans

The loans module contains the following functions *compute_annuity()* and *get_cashflow_schedule()* for pricing a basic consumer loan, plus a couple of utility functions. A functional programming approach is used over an object-oriented approach because simpler is always better, and for computing annuity and cashflow schedule, defining a class is not necessary. Using just pure functions, the risk of side effects from managing changing states is avoided and extension to concurrency is also straightforward.

In [2]:
help(loans.compute_annuity)

Help on function compute_annuity in module loans:

compute_annuity(apr, principal, nterms)
    Computes and returns the annuity of a basic consumer loan.
    
    Args:
        apr: The APR of the loan.
        principal: The initial size of the loan.
        nterms: The number of terms (in month) of the loan.
    
    Returns:
        decimal: A Decimal object representing the monthly annuity.



In [3]:
help(loans.get_cashflow_schedule)

Help on function get_cashflow_schedule in module loans:

get_cashflow_schedule(apr, principal, nterms, date0, dateT)
    Computes and returns a dataframe of cashflow schedule.
    
    Args:
        apr: The APR of the loan.
        principal: The initial size of the loan.
        nterms: The number of terms (in month) of the loan.
        date0: The start date of the loan.
        dateT: The repayment date of the loan.
    
    Returns:
        dataframe: A Pandas object with the cashflow schedule.



The *compute_annuity()* function takes in the APR, principal and number of terms, and returns the monthly annuity using the simple amortised loan equation. The *get_cashflow_schedule()* use *compute_annuity()* to generate a cashflow schedule and returns a Pandas dataframe containing the payment dates, the payment amounts, the principal and interest components of the payments, and the balance. Unfortunately, the current version of the *loans* module generates the cashflow schedule imperatively rather than declaratively, which is to do in the future.

In both functions, all operations are done on the Python *decimal* type instead *float* due to the importance of computing precise values in financial applications. It should be noted that although the decimal type is rather slow in Python 2.7x, in Python 3 Decimal runs on a C backend so it should be much faster.

Using a small SQL database, the functionality of the *loans* module is tested as follows. The SQL database is queried into a Pandas dataframe. To be done in the future is to write a Client class and use SQLAlchemy ORM to map the class to the database.

In [4]:
import sqlite3 as lite
import pandas as pd

con = lite.connect('loans.db')

with con:    
 
    cur = con.cursor()
    
    query = "SELECT * FROM Loans"
    
    df = pd.read_sql(query,con)

In [8]:
for idx,client in df.iterrows():
    cashdf = loans.get_cashflow_schedule(client.APR,client.Principal,client.NumTerms,client.LentDate,client.RepaymentDate)
    
    print('---------------------------------------')
    print('Cashflow for ClientID '+client.ClientID)
    print('---------------------------------------')
    print(cashdf)
    print('\n')

---------------------------------------
Cashflow for ClientID 9dn6RsSnE8Mh
---------------------------------------
   Annuity (£) Interest Component (£) Principal Component (£) Balance (£)  \
0            0                      0                       0        6400   
1       506.46                  26.07                  480.39     5919.61   
2       506.46                  24.12                  482.34     5437.27   
3       506.46                  22.15                  484.31     4952.96   
4       506.46                  20.18                  486.28     4466.68   
5       506.46                  18.20                  488.26     3978.41   
6       506.46                  16.21                  490.25     3488.16   
7       506.46                  14.21                  492.25     2995.91   
8       506.46                  12.21                  494.26     2501.65   
9       506.46                  10.19                  496.27     2005.38   
10      506.46                   8.17 